In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch

from transformers import AutoTokenizer

from nltk.tokenize import word_tokenize
from tqdm import tqdm
import numpy as np
import os

In [ ]:
# set seeds for reproducibility
SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True

In [ ]:
# book collection
BOOKS = {
    'TheOdyssey': "https://raw.githubusercontent.com/cetinsamet/data-science/main/data/book/TheOdyssey_Homer.txt",
    'PrideAndPrejudice': "https://raw.githubusercontent.com/cetinsamet/data-science/main/data/book/PrideAndPrejudice_JaneAusten.txt",
    'AJourneyToTheCentreOfTheEarth': "https://raw.githubusercontent.com/cetinsamet/data-science/main/data/book/JulesVerne_AJourneyToTheCentreOfTheEarth.txt",
}
BOOKS

In [ ]:
# choose a book
BOOK = 'PrideAndPrejudice'

# get the book's url 
book_url = BOOKS[BOOK]
# extract the book's filename from the url
book_filename = os.path.basename(book_url) 

# download the book in local if it doesn't already exist
if not os.path.exists(book_filename):
    try:
        !wget $book_url
        print("File is succesfully downloaded.")
    except Exception as e:
        print(f"Could not download the book from {book_url}")
        print(e)
else:
    print("File has already been downloaded.")

In [ ]:
# define hyperparameters
N_EPOCHS = 30
BATCH_SIZE = 128
SEQ_LEN = 200
LEARNING_RATE = 1e-4
EMBED_SIZE = 128
N_HEADS = 2
NUM_LAYERS = 1
BATCH_FIRST=True
NORM_FIRST=False
TEST_SIZE = 0.2
EARLY_STOPPING_STEP_SIZE = 5
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BEST_MODEL_FILEPATH = 'best_model.pt'

DEVICE

In [ ]:
def load_text(fp):
    with open(fp, mode='r') as infile:
        text = ''.join([row for row in infile])
    return text

# load text
text = load_text(book_filename)

# Get the number of characters in the text
text_len = len(text)
print(f'Number of characters in the text = {text_len}')
print("--------------\n")

# Print a sample from the text
idx = np.random.randint(low=0, high=text_len-SEQ_LEN)
print(f'Sample text:\n{text[idx:(idx + SEQ_LEN)]}')
print("--------------\n")

In [ ]:
# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
# tokenize the whole book
tokens = tokenizer.tokenize(text)
# get the number of tokens
n_tokens = len(tokens)
print(f"There are {n_tokens} tokens in the text.")

# set vocabulary (set of unique tokens in the source text)
#vocab = sorted(list(set(tokens)))
# get the vocabulary size
vocab_size = len(tokenizer)
print(f"Vocabulary size = {vocab_size}")
print("--------------\n")

In [ ]:
# define the train and test data
test_len = int(n_tokens * TEST_SIZE)
train_data = tokens[:-test_len]
test_data = tokens[-test_len:]

# print the length of train and test data
print(f'Length of the train text = {len(train_data)}')
print(f'Length of the test text  = {len(test_data)}')

In [ ]:
# define the TextDataset as an instance of torch.nn.Dataset
class TextDataset(Dataset):
    def __init__(self, text, seq_len):
        super().__init__()
        self.text = text
        self.seq_len = seq_len

    def __len__(self):
        # define the number of samples in the dataset
        return len(self.text) - self.seq_len - 1

    def __getitem__(self, idx):
        # x = [c0, c1, ... cN]
        x = torch.tensor(
            [tokenizer.convert_tokens_to_ids(token) for token in self.text[idx:(idx + self.seq_len)]], 
            dtype=torch.long
        )
        # y = [c1, c1, ... c(N+1)]
        y = torch.tensor(
            [tokenizer.convert_tokens_to_ids(token) for token in self.text[(idx + 1):(idx + 1 + self.seq_len)]], 
            dtype=torch.long
        )
        return x, y

In [ ]:
# initialize train and test datasets
train_dset = TextDataset(train_data, SEQ_LEN)
test_dset = TextDataset(test_data, SEQ_LEN)

# initialize train and test iterators
train_iterator = DataLoader(train_dset, BATCH_SIZE, shuffle=True, drop_last=True, pin_memory=True)
test_iterator = DataLoader(test_dset, BATCH_SIZE, shuffle=False, drop_last=False, pin_memory=True)

In [ ]:
# define GPT network
class GPT(nn.Module):
    def __init__(self, vocab_size, embed_size, seq_len, num_layers, n_heads, batch_first, norm_first):
        super().__init__()
        self.seq_len = seq_len
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(seq_len, embed_size)
        # define transformer encoder network
        encoder_layer = nn.TransformerEncoderLayer(embed_size, N_HEADS, batch_first=batch_first, norm_first=norm_first)
        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)
            
    def forward(self, x, is_causal=False):
        word_embed = self.word_embedding(x)
        pos_embed = self.positional_embedding(torch.arange(x.shape[1], dtype=torch.long).to(x.device))        
        x = word_embed + pos_embed
        if is_causal:
            mask = nn.Transformer.generate_square_subsequent_mask(self.seq_len).to(x.device)
        else: mask=None
        x = self.encoder(x, mask=mask, is_causal=is_causal)
        x = self.fc(x)
        return x

In [ ]:
def compute_loss(logits, y):
    # get the logits dimensions for better readibility and understandibility
    B, S, C = logits.shape
    # compute loss
    loss = F.cross_entropy(logits.view(B*S, C), y.view(-1))
    return loss

def batch_loop(model, x, y, is_causal=False):
    # forward pass
    logits = model(x, is_causal=is_causal)
    # compute loss
    loss = compute_loss(logits, y)
    return loss

def train_epoch(model, iterator):
    # global variable
    global DEVICE
    # set model to training mode
    model.train()
    # initialize epoch loss
    loss_epoch = 0.0
    
    # iterate over batches
    for iter_idx, (x, y) in tqdm(enumerate(iterator), total=len(iterator), desc="Training Progress"):
        # carry tensors to available device
        x, y = x.to(DEVICE), y.to(DEVICE)
        # train batch and compute loss
        loss = batch_loop(model, x, y, is_causal=True)        
        # gradient update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # add batch loss to epoch loss sum
        loss_epoch += loss.item()
        
    # get the number of batches
    n_batch = len(iterator)
    # compute average loss
    loss_epoch_avg = loss_epoch / n_batch
    return loss_epoch_avg

def evaluate(model, iterator):
    # global variable
    global DEVICE
    # set model to evaluation mode
    model.eval()
    # initialize loss
    loss_sum = 0.0
    
    with torch.inference_mode():
        # iterate over batches
        for iter_idx, (x, y) in tqdm(enumerate(iterator), total=len(iterator), desc="Evaluation Progress"):
            # carry tensors to available device
            x, y = x.to(DEVICE), y.to(DEVICE)
            # train batch and compute loss
            loss = batch_loop(model, x, y)        
            # add batch loss to epoch loss sum
            loss_sum += loss.item()
        
    # get the number of batches
    n_batch = len(iterator)
    # compute average loss
    loss_avg = loss_sum / n_batch
    return loss_avg

In [ ]:
def generate(model, text, max_length=100, temp=1.0):
    # global variables
    global DEVICE, tokenizer
    # set model to evaluation mode
    model.eval()
    input_token_ids = [tokenizer.convert_tokens_to_ids(word) for word in tokenizer.tokenize(text)]
    tokens = torch.tensor([input_token_ids], dtype=torch.long)
    generated_token_ids = input_token_ids
    with torch.inference_mode():
        for _ in range(max_length):
            logits = model(tokens)
            next_token_logits = logits[:,-1,:]
            next_token_id = torch.multinomial(next_token_logits.div(temp).exp(), num_samples=1)
            tokens = torch.cat((tokens, torch.tensor([[next_token_id]])), dim=1)
            generated_token_ids.append(next_token_id.item())
    
    return generated_token_ids

In [ ]:
# initialize GPT model
model = GPT(
    vocab_size=vocab_size,
    embed_size=EMBED_SIZE,
    seq_len=SEQ_LEN,
    num_layers=NUM_LAYERS,
    n_heads=N_HEADS,
    batch_first=BATCH_FIRST,
    norm_first=NORM_FIRST,
).to(DEVICE)
model = torch.nn.DataParallel(model)

# initialize optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

best_test_loss = float('inf')
best_loss_streak = 0

# iterate over epochs
for epoch_idx in range(N_EPOCHS):
    train_loss = train_epoch(model, train_iterator)
    test_loss = evaluate(model, test_iterator)
    print(f"Epoch {epoch_idx:02} - Train Loss = {train_loss:.3f}\tTest Loss = {test_loss:.3f}")
    
    # save the current model as the best model if the current test loss is the least achieved 
    if test_loss < best_test_loss:
        # save the curent model's parameters as the best model parameters
        torch.save(model.state_dict(), BEST_MODEL_FILEPATH)
        # replace the best test loss with the current best loss
        best_test_loss = test_loss
        # reset early stoppping counter 
        best_streak_count = 0
        # display info
        print(f'The best model is found and saved. Current best test loss = {best_test_loss:.3f}')
        text = "A"
        generated_tokens = generate(model, text)
        print(tokenizer.decode(generated_tokens))
    else:
        # update early stoppping counter 
        best_streak_count += 1

    # check early stopping condition
    if best_streak_count == EARLY_STOPPING_STEP_SIZE:
        print(f"A better model has not been found in the last {EARLY_STOPPING_STEP_SIZE} epochs. Early stopping...")
        break
        
    print("--------------\n")